In [44]:
import csv
import pandas as pd 
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
import numpy as np

In [45]:
df = pd.read_csv("C:\\Users\\Connor\\Desktop\\Coding\\nft_market_a\\data\\raw_data\\twitter_nft_timeline.csv")
df.head()

,Tweet ID,Username,User ID,Tweet Date,Text
0,1645805247682727938,DegenerateNews,1505874016761831430,2023-04-11 15:05:23+00:00,ICYMI: @OthersideMeta #11260 WITH KODA #9975 S...
1,1645794027399413761,DegenerateNews,1505874016761831430,2023-04-11 14:20:48+00:00,BREAKING: $SOL PUMPS TO $23.11\n\nSOURCE: @Coi...
2,1645789641063415810,DegenerateNews,1505874016761831430,2023-04-11 14:03:22+00:00,ICYMI: @helium TO MIGRATE TO @solana IN 7 DAYS...
3,1645770403187441664,DegenerateNews,1505874016761831430,2023-04-11 12:46:55+00:00,SOURCE: https://t.co/YiyTPsekyd
4,1645770400331120642,DegenerateNews,1505874016761831430,2023-04-11 12:46:55+00:00,BREAKING: SOLANA 24H NFT SALES VOLUME INCREASE...


In [47]:
def process_text(tweet):
    tweet_words = []
    for word in tweet.split(' '):
        if word.startswith('@') and len(word) > 1:
            word = '@user'
        elif word.startswith('http'):
            word = "http"
        tweet_words.append(word)
    tweet_proc = " ".join(tweet_words)
    return tweet_proc

In [48]:
df['processed_text'] = df['Text'].apply(process_text)

print(df['processed_text'])

0     ICYMI: @user #11260 WITH KODA #9975 SOLD FOR 2...
1     BREAKING: $SOL PUMPS TO $23.11\n\nSOURCE: @use...
2       ICYMI: @user TO MIGRATE TO @user IN 7 DAYS http
3                                          SOURCE: http
4     BREAKING: SOLANA 24H NFT SALES VOLUME INCREASE...
                            ...                        
95    BREAKING: A PROJECT ON @user RAISED 921 $ETH (...
96    ICYMI: @user PARTNERED WITH @user TO BRING AN ...
97                                         SOURCE: http
98    BREAKING: @user DISCLOSES THAT @user WAS ONCE ...
99         BREAKING: @user TEASES "HYPERSPACE PRO" http
Name: processed_text, Length: 100, dtype: object


In [49]:
# Load Model
roberta = "cardiffnlp/twitter-roberta-base-sentiment"

model = AutoModelForSequenceClassification.from_pretrained(roberta)

tokenizer = AutoTokenizer.from_pretrained(roberta)

labels = ['Negative', 'Neutral', 'Positive']

In [50]:
# Sentiment analysis

# Initialise a variable to store the total scores
total_scores = np.zeros((1, len(labels)))

for tweet_proc in df['processed_text']:
    encoded_tweet = tokenizer(tweet_proc, return_tensors='pt')
    output = model(encoded_tweet['input_ids'], encoded_tweet['attention_mask'])
    logits = output.logits.detach().numpy()  
    scores = np.exp(logits) / np.sum(np.exp(logits), axis=1, keepdims=True)
    
    # Add the scores to the total scores
    total_scores += scores
    
# Calculate the average scores 
average_scores = total_scores / len(df['processed_text'])

# Find the index of the highest average score
max_index = np.argmax(average_scores)

print(f"{labels[max_index]}: {average_scores[0][max_index]}")

Neutral: 0.7595869036018849
